In [1]:
# Import required packages
import time
from typing import Any
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from paik.solver import Solver
from paik.settings import (
    PANDA_NSF,
    PANDA_PAIK,
    FETCH_PAIK,
    FETCH_ARM_PAIK,
    IIWA7_PAIK,
    ATLAS_ARM_PAIK,
    ATLAS_WAIST_ARM_PAIK,
    BAXTER_ARM_PAIK,
    PR2_PAIK
)
import os
from paik.file import save_pickle, load_pickle

WorldModel::LoadRobot: /home/luca/Klampt-examples/data/robots/baxter.rob
RobParser: Reading robot file /home/luca/Klampt-examples/data/robots/baxter.rob...
RobParser:    Parsing robot file, 54 links read...
LoadAssimp: Loaded model /home/luca/Klampt-examples/data/robots/baxter/collision_head_link_1.off (482 verts, 960 tris)
LoadAssimp: Loaded model /home/luca/Klampt-examples/data/robots/baxter/collision_head_link_2.off (482 verts, 960 tris)
LoadAssimp: Loaded model /home/luca/Klampt-examples/data/robots/baxter/torso.off (39450 verts, 69139 tris)
ManagedGeometry: loaded /home/luca/Klampt-examples/data/robots/baxter/torso.off in time 0.397249s
LoadAssimp: Loaded model /home/luca/Klampt-examples/data/robots/baxter/head.off (2643 verts, 5215 tris)
LoadAssimp: Loaded model /home/luca/Klampt-examples/data/robots/baxter/screen.off (1904 verts, 3486 tris)
LoadAssimp: Loaded model /home/luca/Klampt-examples/data/robots/baxter/display.off (8 verts, 12 tris)
LoadAssimp: Loaded model /home/luca/Kl

In [2]:
param = BAXTER_ARM_PAIK
param.use_dimension_reduction = False
solver = Solver(solver_param=param, load_date="", work_dir="/home/luca/paik")
solver.random_ikp(num_poses=100, num_sols=100)
# solver.generate_ik_solutions(P, F, std, latent)

WorldModel::LoadRobot: /home/luca/Klampt-examples/data/robots/baxter.rob
("[INFO] create new model with config: SolverConfig(robot_name='baxter_arm', "
 "n=7, m=7, r=1, subnet_num_layers=3, model_architecture='nsf', "
 'randperm=False, base_std=0.68, subnet_width=1024, num_transforms=7, '
 'num_bins=10, lr=0.00037, lr_weight_decay=0.012, lr_amsgrad=False, '
 'lr_beta=(0.9, 0.999), noise_esp=0.0025, noise_esp_decay=0.97, '
 'use_dimension_reduction=False, gamma=0.086, batch_size=2048, num_epochs=15, '
 "shce_patience=2, use_nsf_only=False, select_reference_posture_method='knn', "
 "ckpt_name='1118-0317', enable_load_model=True, device='cuda', N=5000000, "
 "data_dir='/home/luca/paik/data/baxter_arm', "
 "train_dir='/home/luca/paik/data/baxter_arm/train', "
 "weight_dir='/home/luca/paik/weights/baxter_arm', max_num_data_hnne=3000000, "
 "traj_dir='/home/luca/paik/data/baxter_arm/trajectory/', "
 "dir_paths=('/home/luca/paik/data/baxter_arm', "
 "'/home/luca/paik/weights/baxter_arm', "
 "

100%|██████████| 2/2 [00:00<00:00,  3.62it/s]


                 l2           ang
count  10000.000000  10000.000000
mean       0.860220    129.849922
std        0.313904     36.419479
min        0.044440      8.890407
25%        0.628855    106.612011
50%        0.878798    136.926532
75%        1.108208    159.469903
max        1.663415    179.994933
  l2 (mm)    ang (deg)    inference_time (ms)
---------  -----------  ---------------------
   860.22       129.85                      9


(0.8602200439572629, 2.266308676389674, 0.009, 0.0)

In [9]:
from hnne import HNNE
J = solver.J
num_data = 10_0000

hnne = HNNE()
Fr = hnne.fit_transform(X=J[:num_data], dim=solver.r, verbose=True)

print(f"[INFO] use_dimension_reduction is False, use clustering.")
partitions = hnne.hierarchy_parameters.partitions
num_clusters = hnne.hierarchy_parameters.partition_sizes
closest_idx_to_num_clusters_20 = np.argmin(
    np.abs(np.array(num_clusters) - 20)
)
Fp = partitions[:, closest_idx_to_num_clusters_20].reshape(-1, 1)

Building h-NNE hierarchy using FINCH...
Using PyNNDescent to compute 1st-neighbours at this step ...
Wed Aug  7 22:28:46 2024 Building RP forest with 23 trees
Wed Aug  7 22:28:47 2024 NN descent for 17 iterations
	 1  /  17
	 2  /  17
	 3  /  17
	Stopping threshold met -- exiting after 3 iterations
Step PyNNDescent done ...
Level 0: 26136 clusters
Level 1: 6342 clusters
Level 2: 1528 clusters
Level 3: 359 clusters
Level 4: 87 clusters
Level 5: 25 clusters
Level 6: 8 clusters
Level 7: 2 clusters
Removing 1 levels from the top to start with a levelof size at least 3.
Overwriting the dimensions 2 to the new value 1.
Projecting to 1 dimensions...
[26136, 6342, 1528, 359, 87, 25, 8]
[INFO] use_dimension_reduction is False, use clustering.


In [13]:
Fp = Fp.reshape(-1, 1)

In [15]:
F = Fp

In [16]:
from sklearn.neighbors import NearestNeighbors
num_total_data = num_data * 2
J = J[:num_total_data]
# query nearest neighbors for the rest of J
if len(F) != len(J):
    knn = NearestNeighbors(n_neighbors=1).fit(J[:num_data])
    F = np.row_stack(
        (
            F,
            F[
                knn.kneighbors(
                    J[num_data:], n_neighbors=1, return_distance=False
                ).flatten()  # type: ignore
            ],
        )
    )  # type: ignore


In [17]:
F.shape

(200000, 1)

In [ ]:
from finch import FINCH
J = solver.J

num_data = min(solver.param.max_num_data_hnne, len(J))

print(f"[INFO] Use FINCH to cluster the posture features.")
cluster_labels_all_partitions, num_clusters, required_clusters = FINCH(
    J[:num_data]
)
closest_idx_to_num_clusters_20 = np.argmin(
    np.abs(np.array(num_clusters) - 20)
)
F = cluster_labels_all_partitions[:, closest_idx_to_num_clusters_20]

df = pd.DataFrame(F)
df.describe()

In [ ]:
from hnne import HNNE
J = solver.J
num_data = min(solver.param.max_num_data_hnne, len(J))

hnne = HNNE(dim=solver.r)
projection = hnne.fit_transform(J[:num_data], verbose=True)

partitions = hnne.hierarchy_parameters.partitions
partition_sizes = hnne.hierarchy_parameters.partition_sizes


In [ ]:
df = pd.DataFrame(projection)
df.describe()

In [ ]:
partitions[:, -2]

In [ ]:
partition_sizes

In [ ]:
closest_idx_to_num_clusters_20 = np.argmin(np.abs(np.array(num_clust) - 20))
closest_idx_to_num_clusters_20

In [ ]:
def oscillate_latent(solver, latent, num_steps=100, step_size=0.1):
    _, P = solver._robot.sample_joint_angles_and_poses(
                n=1, return_torch=False
            )
    F = solver.select_reference_posture(P, "knn")
    print(P.shape, F.shape)
    J_hat = solver.generate_ik_solutions(P, F, num_sols=1, std=0.0, latent=latent)
    assert J_hat.shape == (1, 1, solver.n) # (num_sols, num_poses, n)
    J_hat = np.repeat(np.zeros_like(J_hat), num_steps*2, axis=0) # (num_steps*2, 1, n)
    
    curr_i = 0
    for i in range(num_steps):
        latent[0] += step_size
        J_hat[curr_i] = solver.generate_ik_solutions(P, F, num_sols=1, std=0.0, latent=latent)
        curr_i += 1
        
    for i in range(num_steps):
        latent[0] -= step_size
        J_hat[curr_i] = solver.generate_ik_solutions(P, F, num_sols=1, std=0.0, latent=latent)
        curr_i += 1
        
    return J_hat.reshape(-1, solver.n)
oscillate_latent(solver, np.zeros((7)), num_steps=100, step_size=0.1)

In [ ]:
def oscillate_feature(solver, num_steps=100, step_size=0.1):
    _, P = solver._robot.sample_joint_angles_and_poses(
                n=1, return_torch=False
            )
    F = solver.select_reference_posture(P, "knn")
    J_hat = solver.generate_ik_solutions(P, F, num_sols=1, std=0.0, latent=np.zeros((7)))
    assert J_hat.shape == (1, 1, solver.n) # (num_sols, num_poses, n)
    J_hat = np.repeat(np.zeros_like(J_hat), num_steps*2, axis=0) # (num_steps*2, 1, n)
    
    curr_i = 0
    for i in range(num_steps):
        F += step_size
        J_hat[curr_i] = solver.generate_ik_solutions(P, F, num_sols=1, std=0.0, latent=np.zeros((7)))
        curr_i += 1
        
    for i in range(num_steps):
        F -= step_size
        J_hat[curr_i] = solver.generate_ik_solutions(P, F, num_sols=1, std=0.0, latent=np.zeros((7)))
        curr_i += 1
        
    return J_hat.reshape(-1, solver.n)

oscillate_feature(solver, num_steps=100, step_size=0.1)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def simple_path_planning(solver, P: np.ndarray, num_sols: int):
    base_std = 0.1
    assert len(P.shape) == 2
    num_poses = P.shape[0]
    
    F = solver.select_reference_posture(P[0], "knn", num_sols=num_sols)
    assert F.shape == (num_sols, solver.r)
    P_tr = np.repeat(np.expand_dims(P[0], axis=0), num_sols, axis=0).reshape(-1, P.shape[-1])
    assert P_tr.shape == (num_sols, solver.n)
    J_fr = solver.generate_ik_solutions(P_tr, F, num_sols=1, std=base_std, latent=np.zeros((solver.n)))
    assert J_fr.shape == (1, num_sols, solver.n)
    J_fr = J_fr.reshape(num_sols, solver.n)
    
    if solver._use_nsf_only:
        # feature from the first pose
        F = solver.select_reference_posture(P[np.random.randint(0, num_poses)], "knn")
        F = np.repeat(F, P.shape[0], axis=0)
        J_hat = solver.generate_ik_solutions(P, F, num_sols=num_sols, std=base_std, latent=np.zeros((solver.n)))
    else:
        num_neighbors = 10
        assert num_sols % num_neighbors == 0
        F = solver.select_reference_posture(P, "knn", num_sols=num_neighbors)
        assert F.shape == (num_neighbors*num_poses, solver.r)
        F_tr = F.reshape(num_neighbors, num_poses, solver.r)
        F_tr = np.repeat(np.expand_dims(F_tr, axis=0), num_sols//num_neighbors, axis=0).reshape(-1, solver.r)
        assert F_tr.shape == (num_sols*num_poses, solver.r)
        P_tr = np.repeat(np.expand_dims(P, axis=0), num_sols, axis=0).reshape(-1, P.shape[-1])
        assert P_tr.shape == (num_sols*num_poses, solver.n)
        J_hat = solver.generate_ik_solutions(P_tr, F_tr, num_sols=1, std=base_std, latent=np.zeros((solver.n)))
        assert J_hat.shape == (1, num_sols*num_poses, solver.n)
        J_hat = J_hat.reshape(num_sols, num_poses, solver.n)        
    
    # based on the generated solutions, we can now plan a path
    # we will use the nearest neighbor to the solutions of the previous pose
    # to determine the solution for the next pose
    # we will use the first solution as the initial solution
    def get_nearest_neighbor(next_q_set, curr_q):
        neigh = NearestNeighbors(n_neighbors=1)
        neigh.fit(next_q_set)
        return neigh.kneighbors([curr_q], return_distance=False).flatten()
    
    
    smooth_trajectory = np.zeros_like(J_hat[0])
    
    # random initialization
    smooth_trajectory[0] = J_fr[np.random.randint(0, num_sols), 0]
    
    for i in range(1, num_poses):
        idx = get_nearest_neighbor(J_hat[:, i], smooth_trajectory[i-1])
        smooth_trajectory[i] = J_hat[idx, i]

    # compute maximum joint changes for the smooth trajectory
    max_joint_changes = np.max(np.abs(np.diff(smooth_trajectory, axis=0)))

    return smooth_trajectory.reshape(1, num_poses, solver.n), max_joint_changes

def plan_multiple_trajectories(solver, P, num_trajectories: int, num_samples_per_pose: int):
    num_poses = P.shape[0]
    trajectories = np.zeros((num_trajectories, num_poses, solver.n))
    max_joint_changes = np.zeros(num_trajectories)
    
    for i in range(num_trajectories):
        trajectories[i], max_joint_changes[i] = simple_path_planning(solver, P, num_sols=num_samples_per_pose)
    df = pd.DataFrame(max_joint_changes, columns=["max_joint_changes"])
    print(df.describe())
    
    return trajectories, max_joint_changes
    

poses_function = lambda counter: np.array(
        [0.4 * np.sin(counter / 50), 0.6, 0.75, 0.7071068, -0.7071068, 0.0, 0.0]
    )

num_poses = 10
num_trajectories = 10
num_samples_per_pose = 2000
P = np.array([poses_function(i) for i in range(num_poses)])

# solver = Solver(solver_param=PANDA_PAIK, load_date="0705-0305", work_dir="/home/luca/paik")
trajectories, max_joint_changes = plan_multiple_trajectories(solver, P, num_trajectories=num_trajectories, num_samples_per_pose=num_samples_per_pose)
print(trajectories.shape, max_joint_changes.shape)

nsf = Solver(solver_param=PANDA_NSF, load_date="0115-0234", work_dir="/home/luca/paik")
trajectories, max_joint_changes = plan_multiple_trajectories(nsf, P, num_trajectories=num_trajectories, num_samples_per_pose=num_samples_per_pose)
print(trajectories.shape, max_joint_changes.shape)

In [ ]:
from zuko.distributions import DiagNormal
from zuko.flows import Flow, Unconditional

num_sols = 10
c_rand = torch.ones((1, 9), device=solver._device)
latent = torch.tensor([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2], device=solver._device)

model = Flow(transforms=solver._solver.transforms,  # type: ignore
    base=Unconditional(
        DiagNormal,
        torch.zeros((solver._robot.n_dofs,), device=solver._device) + latent,
        torch.zeros((solver._robot.n_dofs,),
                    device=solver._device) * 0,
        buffer=True,
    ),  # type: ignore
)
model(c_rand).sample((num_sols,))
# model

In [ ]:
solver = Solver(solver_param=PANDA_PAIK, load_date="0703-0717", work_dir="/home/luca/paik")
solver.random_ikp(1000, 100)

In [ ]:
solver = Solver(solver_param=PANDA_NSF, load_date="0115-0234", work_dir="/home/luca/paik")
solver.random_ikp(1000, 100)

In [ ]:
import os
from paik.file import save_pickle, load_pickle

def save_by_date(date: str):
    with open(os.path.join('./', f"{date}.pth"), "w") as f:
        f.write(date)
    
def remove_by_date(date: str):
    os.remove(os.path.join('./', f"{date}.pth"))

def save_if_top3(date: str, l2: float):
    top3_date_path = os.path.join('./', "top3_date.pth")
    if not os.path.exists(top3_date_path):
        save_pickle(top3_date_path, {"date": ["", "", ""], "l2": [1000, 1000, 1000]})
    top3_date = load_pickle(top3_date_path)
    save_idx = -1
    # # if the top3 date has the current date, then check if the current model is better, if so, replace it
    if date in top3_date["date"]:
        if l2 < top3_date["l2"][top3_date["date"].index(date)]:
            save_idx = top3_date["date"].index(date)
    elif l2 < max(top3_date["l2"]):
        save_idx = top3_date["l2"].index(max(top3_date["l2"]))
    
    if save_idx == -1:
        print(f"[INFO] current model is not better than the top3 model in {top3_date_path}")
    else:
        if top3_date["date"][save_idx] != "" and top3_date["date"][save_idx] != date:
            remove_by_date(top3_date["date"][save_idx])
        top3_date["date"][save_idx] = date
        top3_date["l2"][save_idx] = l2
        save_pickle(top3_date_path, top3_date)
        save_by_date(date)
        print(f"[SUCCESS] save the date {date} with l2 {l2:.5f} in {top3_date_path}")
    print(f"[INFO] top3 dates: {top3_date['date']}, top3 l2: {top3_date['l2']}")

In [ ]:
# case 1 sequnece
save_if_top3("0702-1911", 0.10001)
save_if_top3("0702-1914", 0.40004)
save_if_top3("0702-1914", 0.30004)
save_if_top3("0702-1914", 0.30006)
save_if_top3("0702-1913", 0.30003)
save_if_top3("0702-1912", 0.20002)

save_if_top3("0702-1915", 0.00005)
save_if_top3("0702-1916", 0.00006)

In [ ]:
from paik.file import save_pickle, load_pickle
d = load_pickle("/home/luca/paik/weights/panda/top3_date.pth")

In [ ]:
d['date'][1] = ""
d['date'][2] = ""
d['l2'][1] = 1000
d['l2'][2] = 1000
d

In [ ]:
save_pickle("/home/luca/paik/weights/panda/top3_date.pth", d)

In [ ]:
load_pickle("/home/luca/paik/weights/panda/top3_date.pth")

In [ ]:
from paik.model import get_robot

In [ ]:
robot = get_robot(solver_param.robot_name, solver_param.dir_paths)

In [ ]:
robot.n_dofs

In [ ]:
import matplotlib.pyplot as plt
import matplotx
import numpy as np

In [ ]:
fig = plt.figure()  # an empty figure with no Axes
fig, ax = plt.subplots()  # a figure with a single Axes

width = 1
ticks =np.linspace(0, width, 11)

ax.set_xlabel("$\\theta_{1}$")
ax.set_ylabel("$\\theta_{2}$")

plt.xticks(ticks, labels=[i for i in range(len(ticks))])
plt.yticks(ticks, labels=[i for i in range(len(ticks))])

# xtickslabls = ["" for i in range(4)] + ["$\\theta^{max}_{1}$", ""]
# # Set ticks labels for x-axis
# ax.set_xticklabels(xtickslabls)




for n, label in enumerate(ax.xaxis.get_ticklabels()):
    if n != 1 and n != 9:
        label.set_visible(False)

for n, label in enumerate(ax.yaxis.get_ticklabels()):
    if n != 1 and n != 9:
        label.set_visible(False)
        

min_line_num = 1
max_line_num = 8
buffer_width = .5
nbins = 10

lower_bound = (min_line_num-buffer_width)/nbins * width
upper_bound = (max_line_num+buffer_width)/nbins * width
min_line = min_line_num/nbins * width
max_line = max_line_num/nbins * width

print(lower_bound, upper_bound, min_line, max_line)
# line axvline is dashed
plt.axvline(
    x=max_line, ymin=lower_bound, ymax=upper_bound, color="black", linestyle="--", label="theta_1 max"
)

# line axvline is dashed
plt.axvline(
    x=min_line, ymin=lower_bound, ymax=upper_bound, color="black", linestyle="--", label="theta_1 min"
)

# line colour is white
plt.axhline(y=max_line, xmin=lower_bound, xmax=upper_bound, color="black", linestyle="--", label="theta_2 max")

plt.axhline(y=min_line, xmin=lower_bound, xmax=upper_bound, color="black", linestyle="--", label="theta_2 max")


ax.spines['left'].set_position('zero')
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_position('zero')
ax.spines['top'].set_visible(False)
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# make arrows
# ax.plot((1), (0), ls="", marker=">", ms=10, color="k",
#         transform=ax.get_yaxis_transform(), clip_on=False)
# ax.plot((0), (1), ls="", marker="^", ms=10, color="k",
#         transform=ax.get_xaxis_transform(), clip_on=False)
# place legend outside
# plt.legend(bbox_to_anchor=(1.0, 1), loc="upper left")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the limits for the plot
theta1_min, theta1_max = 0, 10
theta2_min, theta2_max = 0, 5

# Create a figure and axis
fig, ax = plt.subplots()

# Set the limits of the plot
ax.set_xlim(theta1_min, theta1_max)
ax.set_ylim(theta2_min, theta2_max)

# Set the labels for the axes
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')

# Draw the grid lines
ax.grid(True, which='both')

# Draw horizontal and vertical lines (grid-like appearance)
for y in np.linspace(theta2_min, theta2_max, 100):
    ax.axhline(y, color='gray', linewidth=0.5, linestyle='--', alpha=0.5)
for x in np.linspace(theta1_min, theta1_max, 100):
    ax.axvline(x, color='gray', linewidth=0.5, linestyle='--', alpha=0.5)

# Example coordinates for the irregular polygons
polygon1 = np.array([[1, 2], [2, 3], [3, 2.5], [2.5, 1.5], [1.5, 1]])
polygon2 = np.array([[6, 3], [7, 4], [8, 3.5], [7.5, 2.5], [6.5, 2]])
polygon3 = np.array([[4, 1], [5, 2], [6, 1.5], [5.5, 0.5], [4.5, 0]])

# Draw the polygons
ax.plot(polygon1[:, 0], polygon1[:, 1], 'ko-')  # Black circles connected by lines
ax.plot(polygon2[:, 0], polygon2[:, 1], 'ko-')
ax.plot(polygon3[:, 0], polygon3[:, 1], 'ko-')

# Draw filled areas (for illustration, using one filled area)
polygon_fill = np.array([[4, 2], [5, 3], [6, 2.5], [5.5, 1.5], [4.5, 1]])
ax.fill(polygon_fill[:, 0], polygon_fill[:, 1], 'gray', alpha=0.5)

# Set the major ticks
ax.set_xticks(np.arange(theta1_min, theta1_max + 1, 1))
ax.set_yticks(np.arange(theta2_min, theta2_max + 1, 1))

# Set the minor ticks
ax.set_xticks(np.arange(theta1_min, theta1_max + 1, 0.2), minor=True)
ax.set_yticks(np.arange(theta2_min, theta2_max + 1, 0.2), minor=True)


# Adding arrow labels to the ends of the axes
ax.annotate(r'$\theta_1$', xy=(theta1_max, theta2_min), xytext=(theta1_max + 0.5, theta2_min - 0.5),
            arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=8),
            fontsize=12, ha='center')
ax.annotate(r'$\theta_2$', xy=(theta1_min, theta2_max), xytext=(theta1_min - 0.5, theta2_max + 0.5),
            arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=8),
            fontsize=12, ha='center')

# Display the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

theta_min = 1.5
theta_max = 8.5
font_size = 14

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(10, 8))

# Set axis labels and limits
# ax.set_xlabel(r'$\theta_1$', fontsize=14)
# ax.set_ylabel(r'$\theta_2$', fontsize=14)
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# Remove tick marks and labels
ax.set_xticks([])
ax.set_yticks([])

# Add min and max labels
ax.text(theta_min, -0.5, r'$\theta_1^{min}$', fontsize=font_size)
ax.text(theta_max, -0.5, r'$\theta_1^{max}$', fontsize=font_size)
ax.text(-0.5, theta_min, r'$\theta_2^{min}$', fontsize=font_size)
ax.text(-0.5, theta_max, r'$\theta_2^{max}$', fontsize=font_size)

# Draw the main curve
t = np.linspace(0, 2*np.pi, 200)
x = 3 + 1*np.cos(t) - .2*np.sin(t-.17) 
y = 6 + 1*np.sin(t) + .2*np.cos(t-.37) 
ax.plot(x, y, 'k-')

# Draw the shaded circle
circle = plt.Circle((6, 4), 1.5, fill=True, facecolor='lightgray', edgecolor='black')
ax.add_artist(circle)

ax.axvline(x=theta_min, color='k', linestyle='--', alpha=0.5)
ax.axvline(x=theta_max, color='k', linestyle='--', alpha=0.5)

ax.axhline(y=theta_min, color='k', linestyle='--', alpha=0.5)
ax.axhline(y=theta_max, color='k', linestyle='--', alpha=0.5)

# Add grid
ax.grid(True, linestyle='--', alpha=0.7)

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)  
ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)

# Extend axis lines
ax.spines['left'].set_bounds(0, 10)
ax.spines['bottom'].set_bounds(0, 10)

# Add axis labels at the ends
ax.text(10.1, 0, r'$\theta_1$', fontsize=font_size, ha='left', va='center')
ax.text(0, 10.1, r'$\theta_2$', fontsize=font_size, ha='center', va='bottom')

# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

randArr = np.random.rand(10, 1)
sortArr = np.sort(randArr, axis=0)

print(f"Original array: \n{randArr}")
print(f"Sorted array: \n{sortArr}")